# Exploratory Data Analysis with AirBnb dataset

Dataset: http://insideairbnb.com/rio-de-janeiro/

**Disciplina Aprendizado Descritivo - UFMG - 2020**

André Correia Lacerda Mafra

Gustavo Germano

Hanna Malaquias

In [333]:
import pandas as pd

## 1) Read datasets

Read dataset containing accomodations attributes

In [334]:
listings = pd.read_csv('data/listings.csv.gz')

/home/off-andre/miniconda3/envs/descritivo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (57,58,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [335]:
listings.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,requires_license,jurisdiction_names,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20200722063547,2020-07-22,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. Please contact ...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/im/pictures/65320518/3...,68997,https://www.airbnb.com/users/show/68997,...,f,NaN,f,moderate,f,f,1,0,0,2.07
1,21280,https://www.airbnb.com/rooms/21280,20200722063547,2020-07-22,Renovated Modern Apt. Near Beach,Immaculately renovated top-floor apartment ove...,This is the best neighborhood in Zona Sul. Fo...,https://a0.muscache.com/im/pictures/60851312/b...,81163,https://www.airbnb.com/users/show/81163,...,f,NaN,f,strict_14_with_grace_period,f,f,1,0,0,1.20
2,25026,https://www.airbnb.com/rooms/25026,20200722063547,2020-07-22,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/im/pictures/3003965/68...,102840,https://www.airbnb.com/users/show/102840,...,f,NaN,f,strict_14_with_grace_period,t,t,3,0,0,1.93


Clean data and keep only useful columns

In [336]:
useful_columns =['host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'guests_included', 'review_scores_rating']

In [337]:
listings = listings[useful_columns]

Read dataset containing reviews (not being used for now)

In [338]:
reviews = pd.read_csv('data/reviews.csv.gz')

In [339]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r\n...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


## 2) Feature Engineering and Preprocessing

### 2.1) Data Analysis

Binary or Continuous features that were aggregated

In [340]:
complex_feature_cols = ['is_superhost', 
                       'accommodates_1_5', 'accommodates_5_10', 'accomodates_10_15', 'accomodates_15_plus',
                       'bath_1', 'bath_2', 'bath_2_5', 'bath_5_plus', 
                       'guests_included_1_2', 'guests_included_2_6', 'guests_included_6_plus'
                      'rating_0_60', 'rating_60_70','rating_70_80', 'rating_80_90', 'rating_90_100']

Print each column's possible values sorted by its frequency

In [341]:
for i in useful_columns:
    print(i)
    print('Qtd ' + str(len(listings[i].unique())))
    print(listings[i].value_counts())
    print('----------------------------------------------')

host_is_superhost
Qtd 3
f    30236
t     5013
Name: host_is_superhost, dtype: int64
----------------------------------------------
neighbourhood_cleansed
Qtd 156
Copacabana           9149
Barra da Tijuca      3929
Ipanema              2949
Jacarepaguá          2028
Botafogo             1755
                     ... 
Gericinó                1
Engenheiro Leal         1
Maré                    1
Campo dos Afonsos       1
Cavalcanti              1
Name: neighbourhood_cleansed, Length: 156, dtype: int64
----------------------------------------------
property_type
Qtd 50
Apartment                            26913
House                                 3542
Condominium                           2047
Loft                                   703
Serviced apartment                     656
Guest suite                            231
Bed and breakfast                      164
Guesthouse                             142
Villa                                  118
Hostel                                 10

### 2.2) Convert to Transactional format

Join feature types and create empty dataframe that will be used as transactional dataset

In [342]:
feature_cols = ['id'] + complex_feature_cols + list(listings['neighbourhood_cleansed'].unique()) + list(listings['room_type'].unique()) + list(listings['property_type'].unique())

In [343]:
len(feature_cols)

227

In [257]:
df_t = pd.DataFrame(columns=feature_cols)

In [258]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Entire serviced apartment,Houseboat,Vacation home,Casa particular (Cuba),Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn


Method for creating empty rows to be filled and appended to df_t

In [259]:
# create empty template for rows
def initialize_rows():
    t_row = {}
    for c in df_t.columns:
        t_row[c] = 0
    return t_row

Append id column to listings so it can be used to generate df_t

In [260]:
listings = listings[['id'] + useful_columns]
listings

,id,host_is_superhost,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,guests_included,review_scores_rating
0,17878,t,Copacabana,Condominium,Entire home/apt,5,1.0,2.0,93.0
1,21280,f,Ipanema,Apartment,Entire home/apt,6,2.0,6.0,97.0
2,25026,f,Copacabana,Apartment,Entire home/apt,2,1.0,2.0,94.0
3,31560,t,Ipanema,Apartment,Entire home/apt,3,1.0,2.0,96.0
4,35636,f,Ipanema,Apartment,Entire home/apt,2,1.5,2.0,94.0
...,...,...,...,...,...,...,...,...,...
35250,44337570,f,Leblon,Apartment,Entire home/apt,4,2.0,1.0,NaN
35251,44337678,f,Copacabana,Apartment,Entire home/apt,4,1.0,1.0,NaN
35252,44338103,f,Freguesia (Jacarepaguá),Apartment,Private room,2,1.5,1.0,NaN
35253,44338273,f,Copacabana,Apartment,Shared room,2,1.5,1.0,NaN


Iterative method to append each listing features to df_t

In [261]:
for index, listing in listings.iterrows():
    
    # simple features
    t_row = initialize_rows()
    t_row['id'] = listing['id']
    t_row['is_superhost'] = 1 if listing['host_is_superhost'] == 't' else 0
    t_row[listing['neighbourhood_cleansed']] == 1
    t_row[listing['property_type']] == 1
    t_row[listing['room_type']] == 1
    
    # complex features
    
    ## accommodations
    if listing['accommodates'] <= 5:
        t_row['accommodates_1_5'] = 1
    elif listing['accommodates'] <= 10:
        t_row['accommodates_5_10'] = 1
    elif listing['accommodates'] <= 15:
        t_row['accommodates_10_15'] = 1
    else:
        t_row['accommodates_15_plus'] = 1
    
    ## bathrooms
    if listing['bathrooms'] == 1:
        t_row['bath_1'] = 1
    elif listing['bathrooms'] == 2:
        t_row['bath_2'] = 1
    elif listing['bathrooms'] <= 5:
        t_row['bath_2_5'] = 1
    else:
        t_row['bath_5_plus'] = 1
         
    ## guests_included
    if listing['guests_included'] <=  2:
        t_row['guests_included_1_2'] = 1
    elif listing['guests_included'] <= 6:
        t_row['guests_included_2_6'] = 1
    else:
        t_row['guests_included_6_plus'] = 1
        
    ## review_scores_rating
    if listing['review_scores_rating'] <= 60:
        t_row['rating_0_60'] = 1
    elif listing['review_scores_rating'] <= 70:
        t_row['rating_60_70'] = 1
    elif listing['review_scores_rating'] <= 80:
        t_row['rating_70_80'] = 1
    elif listing['review_scores_rating'] <= 90:
        t_row['rating_80_90'] = 1
    else:
        t_row['rating_90_100'] = 1
    
    # create listing transaction
    df_t = df_t.append(t_row, ignore_index=True)

In [ ]:
df_t = df_t.fillna(0)

In [262]:
df_t

,id,is_superhost,accommodates_1_5,accommodates_5_10,accomodates_10_15,accomodates_15_plus,bath_1,bath_2,bath_2_5,bath_5_plus,...,Hut,Camper/RV,Dorm,Shared room in serviced apartment,Island,Barn,accommodates_10_15,guests_included_6_plus,accommodates_15_plus,rating_0_60
0,17878,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,21280,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,25026,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,31560,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,35636,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35250,44337570,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35251,44337678,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35252,44338103,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
35253,44338273,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


Saves the dataset

In [265]:
df_t.to_csv('transactional_db.csv')

## 3) Data Mining Algorithms

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
    
http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/

In [296]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from IPython.display import display, HTML
import time

Set id as index and let the columns to be just the items (features)

In [ ]:
df_t = df_t.set_index('id')
df_t

### 3.1) Apriori

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [332]:
ap_freq_items = apriori(df_t, min_support=0.05, use_colnames=True)
ap_freq_items = ap_freq_items.sort_values(by='support', ascending=False).reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):    
    display(ap_freq_items)

,support,itemsets
0,0.896043,(rating_90_100)
1,0.844050,(guests_included_1_2)
2,0.762814,"(guests_included_1_2, rating_90_100)"
3,0.751666,(accommodates_1_5)
4,0.672330,"(accommodates_1_5, rating_90_100)"
5,0.669494,"(guests_included_1_2, accommodates_1_5)"
6,0.601900,"(guests_included_1_2, accommodates_1_5, rating_90_100)"
7,0.525316,(bath_1)
8,0.489462,"(accommodates_1_5, bath_1)"
9,0.475762,"(guests_included_1_2, bath_1)"


### 3.2) Fp-growth

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/

In [331]:
fp_freq_items = fpgrowth(df_t, min_support=0.05, use_colnames=True)
fp_freq_items = fp_freq_items.sort_values(by='support', ascending=False).reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):    
    display(fp_freq_items)

,support,itemsets
0,0.896043,(rating_90_100)
1,0.844050,(guests_included_1_2)
2,0.762814,"(guests_included_1_2, rating_90_100)"
3,0.751666,(accommodates_1_5)
4,0.672330,"(accommodates_1_5, rating_90_100)"
5,0.669494,"(guests_included_1_2, accommodates_1_5)"
6,0.601900,"(guests_included_1_2, accommodates_1_5, rating_90_100)"
7,0.525316,(bath_1)
8,0.489462,"(accommodates_1_5, bath_1)"
9,0.475762,"(guests_included_1_2, bath_1)"


### 3.3) Association Rules

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [315]:
frequent_itemsets = fpgrowth(df_t, min_support=0.05, use_colnames=True)

In [317]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(guests_included_1_2),(rating_90_100),0.844050,0.896043,0.762814,0.903754,1.008605,0.006508,1.080113
1,(rating_90_100),(guests_included_1_2),0.896043,0.844050,0.762814,0.851314,1.008605,0.006508,1.048849
2,(accommodates_1_5),(rating_90_100),0.751666,0.896043,0.672330,0.894453,0.998225,-0.001195,0.984933
3,(rating_90_100),(accommodates_1_5),0.896043,0.751666,0.672330,0.750332,0.998225,-0.001195,0.994657
4,(guests_included_1_2),(accommodates_1_5),0.844050,0.751666,0.669494,0.793192,1.055244,0.035049,1.200790
...,...,...,...,...,...,...,...,...,...
125,"(bath_2_5, accommodates_5_10)",(rating_90_100),0.104638,0.896043,0.095958,0.917051,1.023445,0.002198,1.253258
126,"(bath_2_5, accommodates_5_10)",(guests_included_1_2),0.104638,0.844050,0.073351,0.701003,0.830523,-0.014968,0.521575
127,"(guests_included_1_2, bath_2_5, accommodates_5...",(rating_90_100),0.073351,0.896043,0.068870,0.938902,1.047831,0.003144,1.701472
128,"(accommodates_5_10, bath_2_5, rating_90_100)",(guests_included_1_2),0.095958,0.844050,0.068870,0.717706,0.850312,-0.012124,0.552437


### 3.4) Apriori x Fp-Growth

With higher support itemsets will be smaller, and then apriori can be faster

In [309]:
%timeit -n 100 -r 10 apriori(df_t, min_support=0.6, low_memory=True)

257 ms ± 20.7 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [308]:
%timeit -n 100 -r 10 fpgrowth(df_t, min_support=0.6)

448 ms ± 44.2 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


Smaller supports will imply larget itemsets, thus fpgrowth tends to be faster

In [310]:
%timeit -n 100 -r 10 apriori(df_t, min_support=0.05, low_memory=True)

840 ms ± 98.4 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [311]:
%timeit -n 100 -r 10 fpgrowth(df_t, min_support=0.05)

428 ms ± 96.3 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
